# 🔢 Ref Structures

Primitive Types
- https://learn.microsoft.com/en-us/dotnet/api/system.int32

In [ ]:
void addOne(ref int input) => input++;

int i = 5;
addOne(ref i);
Console.WriteLine(i);

In [ ]:
struct LargeStruct
{
    public long A, B, C, D, E, F, G, H; // 64 bytes
}

void ProcessStruct(LargeStruct s)
{
    // A copy of 's' is created when passed to this method
}

LargeStruct myStruct = new LargeStruct();
ProcessStruct(myStruct); // A copy of 'myStruct' is created
    // data is copied bitwise

// imagine being used in an iterative calculation
// neural network training; we will be allocating and deallocating lot of memory 

- C# 7.2 / .NET 2 Core 2 / Nov 2017
- __ref struct__ instances must be allocated on the stack
- They cannot be boxed, stored in heap-allocated objects (e.g., arrays, class fields), or used in contexts that require heap allocation
- ref struct cannot contain fields that are reference types (e.g., string, arrays, or other classes). This ensures they remain stack-allocated
- Performance oriented data structure
- They cannt outlive the scope in which they are declared

In [ ]:
ref struct Point
{
    public int X;
    public int Y;

    public Point(int x, int y) => (X, Y) = (x, y);
    public int Sum() => X + Y;
}

In [ ]:
readonly struct Point
{
    public readonly int X;          // we can only have readonly fields
    public int Y { get; /*set;*/ }  // canot have a set accessor

    public Point(int x, int y) => (X, Y) = (x, y);
    public int Sum() => X + Y;
}

In [ ]:
readonly ref struct ImmutablePoint
{
    public readonly int X;
    public readonly int Y;

    public ImmutablePoint(int x, int y) => (X, Y) = (x, y);
    public int Sum() => X + Y;
}

__Contiguous Memory Allocation__
- Efficient Data Access: Simpler / Array addressing
- Memory Bandwidth Optimzation: Copy to/from GPU
- Cache Utilization for CPUs and GPUs / TPUs ((Tensor Processing Unit) / NPUs (Neural Processing Unit)
    - Direct Memory Access DMA is used to directly transfer between main memory and accelerator
- Hardware Acceleration
    - GPUs / TPUs / NPUs
- Vectorization: SIMD / Tensor Processing
    - Alignment Requirement: 32/64 even wider alignments for optimized data processing

In [ ]:
ref struct TensorSlice
{
    public Span<float> Data;
    public int[] Dimensions;

    public TensorSlice(Span<float> data, int[] dimensions)
    {
        Data = data;
        Dimensions = dimensions;
    }
}

In [ ]:
using System;

ref struct TensorSlice
{
    public Span<float> Data;
    public int[] Dimensions;

    public TensorSlice(Span<float> data, int[] dimensions)
    {
        Data = data;
        Dimensions = dimensions;
    }

    public TensorSlice SliceDimension(int dimension)
    {
        if (dimension < 0 || dimension >= Dimensions.Length)
            throw new ArgumentOutOfRangeException(nameof(dimension), "Dimension is out of range.");

        // Calculate strides
        int[] strides = new int[Dimensions.Length];
        strides[strides.Length - 1] = 1;
        for (int i = strides.Length - 2; i >= 0; i--)
        {
            strides[i] = strides[i + 1] * Dimensions[i + 1];
        }

        // Calculate the size of the sliced dimension
        int sliceSize = Dimensions[dimension];

        // Calculate the total number of elements in the sliced tensor
        int totalElements = Data.Length / Dimensions[dimension];

        // Create a new span for the sliced data
        Span<float> slicedData = new float[totalElements];

        // Copy the data for the sliced dimension
        for (int i = 0; i < totalElements; i++)
        {
            int index = (i / strides[dimension]) * strides[dimension] * sliceSize + (i % strides[dimension]);
            slicedData[i] = Data[index];
        }

        // Create a new dimensions array for the sliced tensor
        int[] newDimensions = new int[Dimensions.Length - 1];
        for (int i = 0, j = 0; i < Dimensions.Length; i++)
        {
            if (i != dimension)
            {
                newDimensions[j++] = Dimensions[i];
            }
        }

        return new TensorSlice(slicedData, newDimensions);
    }
}

class TensorS
{
    float[] data = { 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12 };
    int[] dimensions = { 3, 4 }; // 3 rows, 4 columns

    public void Compute()
    {
        TensorSlice tensor = new TensorSlice(data, dimensions);

        // Slice the 2nd dimension (columns)
        TensorSlice slicedTensor = tensor.SliceDimension(1);

        // Print the sliced tensor data
        Console.WriteLine("Sliced Tensor Data:");
        foreach (var value in slicedTensor.Data)
            Console.Write(value + " ");
    }
}

new TensorS().Compute();

# 🔢 Native Integers

- C# 9 / .NET 5 / Nov 2020
- nint, nuint
- https://learn.microsoft.com/en-us/dotnet/csharp/language-reference/proposals/csharp-9.0/native-integers

In [ ]:
nint i = 3; // platform dependent
int j = (int)i;
nint k = j;
Console.WriteLine($"i: {i}, j: {j}, k: {k}");

unsafe // why we need unsafe for nint
{
    Console.WriteLine($"{sizeof(int)}, {sizeof(nint)}");
}

# 💾 PostgreSQL Text Search

- https://www.postgresql.org/docs/current/textsearch.html

1. https://www.postgresql.org/docs/17/rules-materializedviews.html
    - Materialized views in PostgreSQL use the rule system like views do, but persist the results in a table-like form

<img src=images/materialized-view.jpg width=700>


2. https://www.postgresql.org/docs/current/gin.html
    - https://en.wikipedia.org/wiki/Inverted_index
    - Generalized Inverted Index is designed for handling cases where the items to be indexed are composite values, and the queries to be handled by the index need to search for element values that appear within the composite items. For example, the items could be documents, and the queries could be searches for documents containing specific words

<img src=images/inverted-index.webp width=700><br>
<img src=images/inverted-index.png width=700>

3. https://www.postgresql.org/docs/current/datatype-textsearch.html
    - A tsvector value is a sorted list of distinct lexemes, which are words that have been normalized to merge different variants of the same word

<img src=images/tsvector.png height=500>

- https://www.youtube.com/watch?v=NPduWiPzhpE Full Text Search in .NET With PostgreSQL and EF Core

# 🧮 Vector Databases

## Thinking about Vector Data

<img src=images/data-structures.png width=700><br>
<img src=images/data-modeling.png width=700><br>
<img src=images/data-modeling-1000ft.jpg width=700>

<img src=images/data-access.jpg width=700>

- Optionally Encode/Decode data (Xml, Json, Encryption)
- Querying Database using DSL
- Fetch related data in memory and do processing

<img src=images/embeddings-model.png width=700>

- Vectorization (usually through Neural Network / Model)
    - Choosing Neural Network / Model
- Querying Database using DSL
- Fetch related data in memory and do processing

## Vector Databases in AI/ML

<img src=images/agentic-rag.png height=500>

- https://vectorize.io/how-i-finally-got-agentic-rag-to-work-right

- https://devblogs.microsoft.com/dotnet/announcing-dotnetconf-focus-on-ai
- https://devblogs.microsoft.com/dotnet/azure-ai-model-catalog-dotnet-inference-sdk
- https://devblogs.microsoft.com/dotnet/introducing-pinecone-dotnet-sdk 👈
- https://devblogs.microsoft.com/dotnet/introducing-microsoft-extensions-ai-preview
- https://devblogs.microsoft.com/dotnet/e-shop-infused-with-ai-comprehensive-intelligent-dotnet-app-sample
- https://devblogs.microsoft.com/dotnet/introducing-microsoft-extensions-vector-data 👈
- https://devblogs.microsoft.com/dotnet/local-ai-models-with-dotnet-aspire
- https://devblogs.microsoft.com/dotnet/vector-data-qdrant-ai-search-dotnet 👈

<img src=images/pinecore-vs-qdrant.png>

## 🧮 Qdrant

1. Qdrant Client .NET Example
2. Microsoft.Extensions.VectorData Example
3. Semantic Kernel Memory Example
4. Semantic Kernel Search Plugin Example

- https://qdrant.tech/documentation/quickstart 👈

In [1]:
docker run -p 6333:6333 -p 6334:6334 qdrant/qdrant

In [ ]:
#!connect jupyter --kernel-name pythonkernel --kernel-spec python3

In [ ]:
!pip install qdrant_client
!pip install numpy

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

client = QdrantClient(url="http://localhost:6333")

client.create_collection(
    collection_name="test_collection",
    vectors_config=VectorParams(size=4, distance=Distance.DOT),
)

operation_info = client.upsert(
    collection_name="test_collection",
    wait=True,
    points=[
        PointStruct(id=1, vector=[0.05, 0.61, 0.76, 0.74], payload={"city": "Berlin"}),
        PointStruct(id=2, vector=[0.19, 0.81, 0.75, 0.11], payload={"city": "London"}),
        PointStruct(id=3, vector=[0.36, 0.55, 0.47, 0.94], payload={"city": "Moscow"}),
        PointStruct(id=4, vector=[0.18, 0.01, 0.85, 0.80], payload={"city": "New York"}),
        PointStruct(id=5, vector=[0.24, 0.18, 0.22, 0.44], payload={"city": "Beijing"}),
        PointStruct(id=6, vector=[0.35, 0.08, 0.11, 0.44], payload={"city": "Mumbai"}),
    ],
)

print(operation_info)

In [ ]:
search_result = client.query_points(
    collection_name="test_collection", query=[0.05, 0.61, 0.76, 0.74], limit = 3
    # collection_name="test_collection", query=[0.2, 0.1, 0.9, 0.7], limit = 3
).points

print(search_result)

In [ ]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

search_result = client.query_points(
    collection_name="test_collection",
    query=[0.2, 0.1, 0.9, 0.7],
    query_filter=Filter(
        must=[FieldCondition(key="city", match=MatchValue(value="London"))]
    ),
    with_payload=True,
    limit=3,
).points

print(search_result)

- https://qdrant.tech/documentation/interfaces/web-ui
- https://qdrant.tech/documentation/tutorials
    - https://qdrant.tech/documentation/tutorials/search-beginners 👈
- https://qdrant.tech/documentation/examples 👈
- https://devblogs.microsoft.com/dotnet/vector-data-qdrant-ai-search-dotnet 👈
- https://github.com/Azure-Samples/openai/tree/main/End_to_end_Solutions/GithubRepoAssistant 👈

## 🧮 PostgreSQL

- https://github.com/pgvector/pgvector
- https://www.citusdata.com
- https://learn.microsoft.com/en-us/azure/architecture/databases/idea/intelligent-apps-using-azure-database-for-postgresql
- https://microsoft.github.io/kernel-memory/extensions/memory-db/postgres

To quickly have pgvector; use Docker and run it using one of the following command
- docker run -e POSTGRES_PASSWORD=uworx -p 5432:5432 -d ankane/pgvector:latest
- docker run -e POSTGRES_PASSWORD=uworx -p 5432:5432 -d pgvector/pgvector:pg17 👈

In [ ]:
create extension vector;
select * from pg_extension;

create table Items (Id bigserial primary key, embedding vector(3));
insert into Items (embedding) values ('[1, 2, 3]'), ('[4, 5, 6]');

select * from Items order by embedding <-> '[3,1,2]' limit 5;
/*
    <-> is L2 distance, <#> is inner product, <=> is cosine distance
*/

__Further Readings__
- https://learn.microsoft.com/en-us/azure/postgresql/flexible-server/how-to-use-pgvector

# 🏁

## 🔣 Models

__Text Embedding Models__
- all-minilm, all-MiniLM-L6-v2
    - Best for lightweight, fast, and general-purpose use cases where resource efficiency is critical
- nomic-embed-text
    - A balanced choice for general-purpose tasks with a focus on transparency and ethical AI
- mxbai-embed-text, nomic-embed-large
    - Ideal for high-performance, multilingual, and complex tasks where accuracy and richness of embeddings are prioritized over speed and resource usage

<img src=images/ollama-ls.png width=800>

## 🏁🏁

- Clinic RAG Scenario
    - https://github.com/khurram-uworx/dotnetdebrief/blob/main/src/ChatBots/KernelMemoryQdrantRagSK.cs
    - https://github.com/khurram-uworx/dotnetdebrief/blob/main/src/ChatBots/KernelMemoryPgRagSK.cs
    - Llama cleverness providing fake data for function calls; Mistral was working as expected

<img src=images/sk-tool-km.png>

# 📚 Resources

- https://github.com/khurram-uworx/dotnetdebrief/blob/main/src/ChatBots

__eShop__
- https://github.com/dotnet/eShop is a reference .NET application implementing an eCommerce site / microservices - containers and what not
    - https://github.com/dotnet-architecture

<img src=images/eshop-running.png><br>
<img src=images/eshop-architecture.png width=900><br>

__eShop Support__
- https://devblogs.microsoft.com/dotnet/e-shop-infused-with-ai-comprehensive-intelligent-dotnet-app-sample
- https://www.youtube.com/watch?v=yMGTUQhjtlM Enhancing Business Processes with .NET Aspire and Generative AI
- https://github.com/dotnet/eShopSupport

__.NET AI Samples__
- https://github.com/dotnet/ai-samples

__Azure Samples__
- https://github.com/Azure-Samples
- https://github.com/Azure-Samples/azure-ai A hub with a curated awesome list of all Azure AI samples
    - https://github.com/Azure-Samples/openai
    - https://github.com/Azure-Samples/azureai-samples

__Microsoft Samples__
- https://github.com/microsoft/samples-for-ai
- https://learn.microsoft.com/en-us/windows/ai/samples